# Introduction

In this notebook, we will explore how to use AIF360 and AIX360 tool in creating an application for mortage approval

Sections:
    
    1. File Read
    2. Bias check and mitigation
    3. Evaluate similar applicant
    4. Contrastive explanations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# 
from sklearn import preprocessing
from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import Dense
from IPython.display import Markdown, display
from sklearn.model_selection import train_test_split
import tensorflow as tf
#
import aif360
import aix360
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aix360.algorithms.protodash import ProtodashExplainer
from aix360.algorithms.contrastive import CEMExplainer, KerasClassifier
#
import warnings
warnings.filterwarnings("ignore")

# File Read

In [ ]:
data = pd.read_csv('preprocessed_data.csv')

In [ ]:
data.head()

# AIF360

In [ ]:
# Sex
# 0 - Male 
# 1 - Female 
# Action Taken
# 0 - Application denied 
# 1 - Purchased Loan 

In [ ]:
privileged_groups = [{'applicant_sex': 0}]
unprivileged_groups = [{'applicant_sex': 1}]
favorable_label = 1 
unfavorable_label = 0

In [ ]:
data.info()

In [ ]:
BM_dataset = BinaryLabelDataset(favorable_label=1,
                                unfavorable_label=0,
                                df=data,
                                label_names=['action_taken'],
                                protected_attribute_names=['applicant_sex'],
                                unprivileged_protected_attributes=[np.array([1])],
                                privileged_protected_attributes=[np.array([0])])

In [ ]:
display(Markdown("#### Training Data Details"))
print("shape of the training dataset", BM_dataset.features.shape)
print("Training data favorable label", BM_dataset.favorable_label)
print("Training data unfavorable label", BM_dataset.unfavorable_label)
print("Training data protected attribute", BM_dataset.protected_attribute_names)
print("Training data privileged protected attribute (1:Male and 0:Female)", 
      BM_dataset.privileged_protected_attributes)
print("Training data unprivileged protected attribute (1:Male and 0:Female)",
      BM_dataset.unprivileged_protected_attributes)

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(BM_dataset, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % 
      metric_orig_train.mean_difference())

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(BM_dataset)
train_tf_dataset = RW.transform(BM_dataset)

In [ ]:
metric_orig_train_updated = BinaryLabelDatasetMetric(train_tf_dataset, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Difference in mean outcomes between unprivileged and privileged groups = %f"
      % metric_orig_train_updated.mean_difference())

# AIX360

In [ ]:
dfTrain, dfTest, yTrain, yTest = train_test_split(train_tf_dataset.features, train_tf_dataset.labels, random_state=0)

In [ ]:
dfTrain.shape

In [ ]:
dfTest.shape

In [ ]:
# Simple neural network for the classification task
def nn_small():
    model = Sequential()
    model.add(Dense(500, input_dim=14, activation='tanh'))
    model.add(Dense(500, activation='relu'))
    model.add(Dense(2))    
    return model

In [ ]:
# loss function
def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct, logits=predicted)

nn = nn_small()
nn.compile(loss=fn, optimizer='adam', metrics=['accuracy'])
nn.summary()

In [ ]:
# To train uncomment this cell

# nn.fit(dfTrain, yTrain, batch_size=32, epochs=10, verbose=1, shuffle=False)

# nn.save_weights("trained_model.h5")     

In [ ]:
### Load trained model weights 
nn.load_weights("trained_model.h5")

### Display similar applicant user profiles and the extent to which they are similar to the chosen applicant as indicated by the last row in the table below labelled as "Weight".

##### Explainer used:

ProtodashExplainer provides exemplar-based explanations for summarizing datasets as well as explaining predictions made by an AI model. It employs a fast gradient based algorithm to find prototypes along with their (non-negative) importance weights. The algorithm minimizes the maximum mean discrepancy metric and has constant factor approximation guarantees for this weakly submodular function.


More Details: 
https://aix360.readthedocs.io/en/latest/die.html#protodash-explainer

In [ ]:
p_train = nn.predict_classes(dfTrain) # Use trained neural network to predict train points
p_train = p_train.reshape((p_train.shape[0],1))

z_train = np.hstack((dfTrain, p_train)) # Store (normalized) instances that were predicted as Accepted
z_train_good = z_train[z_train[:,-1]==1, :]

In [ ]:
idx = 86

class_names = ['Denied', 'Accepted']

X = dfTest[idx].reshape((1,) + dfTest[idx].shape)

print("Chosen Sample:", idx)
print("Prediction made by the model:", class_names[np.argmax(nn.predict_proba(X))])
print("Prediction probabilities:", nn.predict_proba(X))
print("")

# attach the prediction made by the model to X
X = np.hstack((X, nn.predict_classes(X).reshape((1,1))))

Xun = dfTest[idx].reshape((1,) + dfTest[idx].shape) 
dfx = pd.DataFrame.from_records(Xun.astype('double')) # Create dataframe with original feature values
dfx[14] = class_names[int(X[0, -1])]
dfx.columns = data.columns
dfx.transpose()

In [ ]:
explainer = ProtodashExplainer()
(W, S, setValues) = explainer.explain(X, z_train_good, m=2) # Return weights W, Prototypes S and objective function values

In [ ]:
dfs = pd.DataFrame.from_records(z_train_good[S, 0:-1].astype('double'))
RP=[]
for i in range(S.shape[0]):
    RP.append(class_names[int(z_train_good[S[i], -1])]) # Append class names
dfs[14] = RP
dfs.columns = data.columns  
dfs["Weight"] = np.around(W, 2)/np.sum(np.around(W, 2)) # Calculate normalized importance weights
dfs.transpose()

# Contrastive explanations

Example: when people ask for an explanation of an event -- the fact --- they (sometimes implicitly) are asking for an explanation relative to some contrast case; that is, "Why P rather than Q?".

We now demonstrate how to compute contrastive explanations using AIX360 and how such explanations can help applicant understand the decisions made by AI models that approve or reject their applications.

In this context, contrastive explanations provide information to applicants about what minimal changes to their profile would have changed the decision of the AI model from reject to accept or vice-versa (pertinent negatives).

Terms:

##### 1. Pertinent Negatives (PN) 
    
   PNs identify a minimal set of features which if altered would change the classification of the original input. 
   
   
##### 2. Pertinent Positives (PP) : 

  PPs on the other hand identify a minimal set of features and their values that are sufficient to yield the original   input's classification. 

  For example, for an applicant whose HELOC application was approved, the explanation may say that even if the         number of satisfactory trades was reduced to a lower number, the loan would have still gotten through.

## Let's explore PP

In [ ]:
# Some interesting user samples to try: 8 9 11
idx = 10

X = dfTest[idx].reshape((1,) + dfTest[idx].shape)
print("Computing PP for Sample:", idx)
print("Prediction made by the model:", class_names[np.argmax(nn.predict_proba(X))])
print("Prediction probabilities:", nn.predict_proba(X))
print("")


mymodel = KerasClassifier(nn)
explainer = CEMExplainer(mymodel)

arg_mode = 'PP' # Find pertinent positives
arg_max_iter = 1000 # Maximum number of iterations to search for the optimal PN for given parameter settings
arg_init_const = 10.0 # Initial coefficient value for main loss term that encourages class change
arg_b = 9 # No. of updates to the coefficient of the main loss term
arg_kappa = 0.1 # Minimum confidence gap between the PNs (changed) class probability and original class' probability
arg_beta = 1e-1 # Controls sparsity of the solution (L1 loss)
arg_gamma = 100 # Controls how much to adhere to a (optionally trained) auto-encoder
my_AE_model = None # Pointer to an auto-encoder

(adv_pp, delta_pp, info_pp) = explainer.explain_instance(X, arg_mode, my_AE_model, arg_kappa, arg_b,
                                                         arg_max_iter, arg_init_const, arg_beta, arg_gamma)

In [ ]:
Xpp = delta_pp
classes = [ class_names[np.argmax(nn.predict_proba(X))], class_names[np.argmax(nn.predict_proba(Xpp))]]

print("PP for Sample:", idx)
print("Prediction(Xpp) :", class_names[np.argmax(nn.predict_proba(Xpp))])
print("Prediction probabilities for Xpp:", nn.predict_proba(Xpp))
print("")


Xpp_re = X - adv_pp
Xpp_re = np.around(Xpp_re.astype(np.double), 2)
Xpp_re[Xpp_re < 1e-4] = 0

X2 = np.vstack((X, Xpp_re))

dfpp = pd.DataFrame.from_records(X2.astype('double')) # Showcase a dataframe for the original point and PP
dfpp[23] = classes
dfpp.columns = data.columns
dfpp.rename(index={0:'X',1:'X_PP'}, inplace=True)
dfppt = dfpp.transpose()

def highlight_ce(s, col, ncols):
    if (type(s[col]) != str):
        if (s[col] > 0):
            return(['background-color: yellow']*ncols)    
    return(['background-color: white']*ncols)

dfppt.style.apply(highlight_ce, col='X_PP', ncols=2, axis=1)

In [ ]:
plt.rcdefaults()
fi = abs(Xpp_re.astype('double'))/np.std(dfTrain.astype('double'), axis=0) # Compute PP feature importance
    
objects = data.columns[-2::-1]
y_pos = np.arange(len(objects)) # Get input feature names
performance = fi[0, -1::-1]

plt.barh(y_pos, performance, align='center', alpha=0.5) # Bar chart
plt.yticks(y_pos, objects) # Plot feature names on y-axis
plt.xlabel('weight') #x-label
plt.title('PP (feature importance)') # Figure heading

plt.show()    # Display the feature importance